In [3]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("mini-contest/dataset-agent.csv")

# Display the first few rows of the DataFrame
print(df.head(1))

   id                                          project_a  \
0   2  https://github.com/prettier-solidity/prettier-...   

                                    project_b  weight_a  weight_b  \
0  https://github.com/nomicfoundation/hardhat  0.101669  0.898331   

   agent_weight_a  agent_weight_b  
0            0.85            0.15  


In [5]:
# force a fresh import of the module, otherwise you need to restart the ipython kernel whenever you make changes to the deepfunding.py file
import importlib
import deepfunding
importlib.reload(deepfunding)

from deepfunding import run_comparison
from tools.data_collection import fetch_repo_metrics

# Initialize agent weight columns if they don't exist
if 'agent_weight_a' not in df.columns:
    df['agent_weight_a'] = None
if 'agent_weight_b' not in df.columns:
    df['agent_weight_b'] = None

for i in range(len(df)):
    if pd.notnull(df.at[i, 'agent_weight_a']) and pd.notnull(df.at[i, 'agent_weight_b']):
        continue

    repo_a_url = df.iloc[i]["project_a"]
    repo_b_url = df.iloc[i]["project_b"]
    print(f"{repo_a_url} VS {repo_b_url}")
    repo_a = { "url": repo_a_url, **fetch_repo_metrics(repo_a_url) }
    repo_b = { "url": repo_b_url, **fetch_repo_metrics(repo_b_url) }

    agent_result = run_comparison(repo_a, repo_b)

    df.at[i, 'agent_weight_a'] = agent_result['weights'][repo_a_url]
    df.at[i, 'agent_weight_b'] = agent_result['weights'][repo_b_url]
    
    print(f"Agent: {df.iloc[i]['agent_weight_a']} {df.iloc[i]['agent_weight_b']}")
    print(f"Human: {df.iloc[i]['weight_a']} {df.iloc[i]['weight_b']}")
    df.to_csv("mini-contest/dataset-agent.csv", index=False)

https://github.com/walletconnect/walletconnect-monorepo VS https://github.com/sheetjs/js-crc32
Project Analyzer Result: weight_a=0.8 weight_b=0.2 confidence=0.85 reasoning="Repo A shows a high level of activity, with 30 active developers and a significant number of commits (1954 in 6 months), indicating ongoing development and community support. The high star count (9940) and large contributor count (2869) suggest strong interest and utility in the project. However, it has a concerning number of closed issues (212) and opened pull requests (896), indicating some challenges in managing contributions. The total funding received is also a factor, with a substantial amount of $136,708, though it hasn't received funding in the last 6 months. The project is more mature and has better visibility, suggesting it is likely to attract new contributors despite its issues.\n\nIn contrast, Repo B has no active developers and no recent commits, indicating it is effectively stagnant (0 commit count in

Request GET /repos/WalletConnect/walletconnect-monorepo/readme failed with 403: rate limit exceeded
Setting next backoff to 2587.25849s


KeyboardInterrupt: 